In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, ConvBertForSequenceClassification, ConvBertTokenizer
from transformers import AutoTokenizer

# Load dataset
# df = pd.read_csv('datasets/cleaned_OLID_test.tsv', sep="\t")
# df = pd.read_csv('datasets/cleaned_OLID.tsv', sep="\t")
df = pd.read_csv('datasets/cleaned_SOLIDtest6K_trainer.tsv', sep='\t')
# df = pd.read_csv('datasets/cleaned_tr_offenseval_test.tsv', sep='\t')
# df = pd.read_csv('datasets/cleaned_hatespeech_offensive.tsv', sep='\t')

# Assuming your columns are named 'tweet' and 'class', change accordingly
tweets = df['tweet'].values
labels_df = df['label'].values


# Split the dataset into training and validation sets
# _, tweets, _, labels_df = train_test_split(tweets, labels_df, test_size=0.2, random_state=42)

# val_texts = tweets
# Initialize BERT tokenizer

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# tokenizer = ConvBertTokenizer.from_pretrained('YituTech/conv-bert-base')
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased") ## bu ve alttaki değişiyor, birde buna göre importlar değişir

# tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")

# Tokenize and encode the training and validation texts
# train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
encodings = tokenizer(tweets.tolist(), truncation=True, padding=True)


/home/cymn/miniconda3/envs/nvidia-newest/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(encodings, labels_df)

# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=12, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=12, shuffle=False)


In [3]:
import torch
from torch import optim
from transformers import BertForSequenceClassification, DistilBertForSequenceClassification
# from transformers import AutoTokenizer, RobertaForSequenceClassification
import time
# Initialize BERT model for sequence classification
# model = RobertaForSequenceClassification.from_pretrained('models/SOLID_semi_RoBERTa_2_cleaned')
# model = BertForSequenceClassification.from_pretrained('models/troffenseval_SOLIDBERT')
# model = ConvBertForSequenceClassification.from_pretrained('models/troffenseval_SOLIDconvBERT', num_labels=2)
model = DistilBertForSequenceClassification.from_pretrained("models/OLIDdistilBERT_1")

# Define optimizer and learning rate
optimizer = optim.AdamW(model.parameters(), lr=1e-5)

# Training loop
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [4]:
import numpy as np
import time
# Evaluation
model.eval()
# Perform evaluation on validation set and calculate metrics as needed
# Example: calculate accuracy
correct = 0
total = 0
i = 0
prediction_list = np.array([])

with torch.no_grad():
    test_start  = time.time()
    print('start')
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)
        prediction_list = np.append(prediction_list, predictions.detach().cpu().numpy())
    print('end')
    
    test_end = time.time()

accuracy = correct / total
print(f'Validation Accuracy: {accuracy}')


start
end
Validation Accuracy: 0.5014183213749375


In [5]:
# wrong_indices = [i for i in range(len(labels_df)) if labels_df[i] != prediction_list[i]]

# output_results_df = pd.DataFrame(columns=['index', 'tweet', 'prediction', 'label'])

# for index in wrong_indices:
#     # output_results_df
    
#     output_results_df.loc[len(output_results_df)] = [index, tweets[index], int(prediction_list[index]), labels_df[index]]
#     # pd.concat([output_results_df, {'tweet': tweets[index], 'prediction': prediction_list[index], 'label': labels_df[index]}])
#     # print(f'tweet: {tweets[index]}, prediction {prediction_list[index]}, label {labels_df[index]:f02}')

# output_results_df.to_csv('./outputs/wr-troffenseval_SOLID_BERT.tsv', sep='\t')

In [6]:
from sklearn.metrics import classification_report

# Assuming you have the true labels in `val_labels` and the predicted labels in `prediction_list`
report = classification_report(labels_df, prediction_list)

print(report)

              precision    recall  f1-score   support

           0       1.00      0.00      0.00      2991
           1       0.50      1.00      0.67      3002

    accuracy                           0.50      5993
   macro avg       0.75      0.50      0.33      5993
weighted avg       0.75      0.50      0.34      5993



In [7]:
from sklearn.metrics import confusion_matrix

# Assuming you have the true labels in `val_labels` and the predicted labels in `prediction_list`
cm = confusion_matrix(labels_df, prediction_list)

# Extract TP, TN, FP, FN from the confusion matrix
TP = cm[1, 1]
TN = cm[0, 0]
FP = cm[0, 1]
FN = cm[1, 0]

print(f"True Positives (TP): {TP}")
print(f"True Negatives (TN): {TN}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")


True Positives (TP): 3002
True Negatives (TN): 3
False Positives (FP): 2988
False Negatives (FN): 0


In [8]:
test_time = test_end - test_start

print(f'test time: {test_time}')

test time: 7.286987066268921


In [9]:
# import pandas as pd

# # Create a DataFrame with the validation texts and labels
# validation_df = pd.DataFrame({'text': val_texts, 'label': val_labels})

# # Add the prediction list as a new column to the DataFrame
# validation_df['prediction'] = prediction_list

# # Save the DataFrame as a CSV file
# validation_df.to_csv('OLID_validation_with_predictions.csv', index=False)
